In [ ]:
import copy
from heapq import heappush, heappop

n = 3

rows = [1, 0, -1, 0]
cols = [0, -1, 0, 1]

class priorityQueue:
    def __init__(self):
        self.heap = []

    def push(self, key):
        heappush(self.heap, key)

    def pop(self):
        return heappop(self.heap)

    def empty(self):
        return not self.heap


class nodes:
    def __init__(self, parent, mats, empty_tile_posi, costs, levels):
        self.parent = parent
        self.mats = mats
        self.empty_tile_posi = empty_tile_posi
        self.costs = costs
        self.levels = levels

    def __lt__(self, nxt):
        return self.costs < nxt.costs


def calculateCosts(mats, final):
    count = 0
    for i in range(n):
        for j in range(n):
            if mats[i][j] and mats[i][j] != final[i][j]:
                count += 1
    return count


def newNodes(mats, empty_tile_posi, new_empty_tile_posi, levels, parent, final):
    new_mats = copy.deepcopy(mats)

    x1, y1 = empty_tile_posi
    x2, y2 = new_empty_tile_posi
    new_mats[x1][y1], new_mats[x2][y2] = new_mats[x2][y2], new_mats[x1][y1]

    costs = calculateCosts(new_mats, final)

    return nodes(parent, new_mats, new_empty_tile_posi, costs, levels)


def printMatsrix(mats):
    for i in range(n):
        for j in range(n):
            print("%d " % mats[i][j], end=" ")
        print()


def isSafe(x, y):
    return 0 <= x < n and 0 <= y < n


def printPath(root):
    if root is None:
        return
    printPath(root.parent)
    printMatsrix(root.mats)
    print()


def solve(initial, empty_tile_posi, final):
    pq = priorityQueue()

    costs = calculateCosts(initial, final)
    root = nodes(None, initial, empty_tile_posi, costs, 0)
    pq.push(root)

    while not pq.empty():
        minimum = pq.pop()

        if minimum.costs == 0:
            printPath(minimum)
            return

        for i in range(4):
            new_tile_posi = [
                minimum.empty_tile_posi[0] + rows[i],
                minimum.empty_tile_posi[1] + cols[i],
            ]

            if isSafe(new_tile_posi[0], new_tile_posi[1]):
                child = newNodes(
                    minimum.mats,
                    minimum.empty_tile_posi,
                    new_tile_posi,
                    minimum.levels + 1,
                    minimum,
                    final,
                )
                pq.push(child)


initial = [
    [1, 2, 3],
    [5, 6, 0],
    [7, 8, 4],
]

final = [
    [1, 2, 3],
    [5, 8, 6],
    [0, 7, 4],
]

empty_tile_posi = [1, 2]

solve(initial, empty_tile_posi, final)


Trạng thái: một ma trận 3×3 (mats)

Ô trống: giá trị 0

Mỗi trạng thái = một node trong cây tìm kiếm

Mục tiêu: biến initial → final

 **Heuristic của AKT trong code**
def calculateCosts(mats, final):


Heuristic h(n) = số ô sai vị trí so với trạng thái đích

Không tính ô trống

Giá trị này được lưu trong costs

Heuristic càng nhỏ → trạng thái càng “gần đích”

class priorityQueue


Thuật toán dùng hàng đợi ưu tiên

Mỗi lần luôn lấy ra node có costs nhỏ nhất

def __lt__(self, nxt):
    return self.costs < nxt.costs

cơ chế “cận”:chỉ mở rộng nhánh có heuristic tốt nhất, các nhánh “xấu” bị trì hoãn.

** Cách sinh trạng thái mới **
for i in range(4):


Ô trống được di chuyển theo 4 hướng: xuống, trái, lên, phải

Nếu vị trí hợp lệ (isSafe) → sinh trạng thái con

Mỗi trạng thái con:

Sao chép ma trận

Hoán đổi ô trống

Tính lại heuristic

Gắn parent

** Cách thuật toán hoạt động trong solve()**

Đưa trạng thái ban đầu vào hàng đợi

Lặp:

Lấy trạng thái có heuristic nhỏ nhất

Nếu costs == 0 → đã tới trạng thái đích

Nếu chưa:

Sinh các trạng thái con

Đưa vào hàng đợi

Truy vết ngược bằng parent để in lời giải

Thuật toán không duyệt toàn bộ,mà luôn “lao” về phía trạng thái có vẻ gần đích nhất.


In [4]:
from collections import deque

class Graph:
    def __init__(self, adjac_lis):
        self.adjac_lis = adjac_lis

    def get_neighbors(self, v):
        return self.adjac_lis[v]

    def h(self, n):
        H = {
            'A': 1,
            'B': 1,
            'C': 1,
            'D': 1
        }
        return H[n]

    def a_star_algorithm(self, start, stop):
        open_lst = set([start])
        closed_lst = set([])

        poo = {}
        poo[start] = 0

        par = {}
        par[start] = start

        while len(open_lst) > 0:
            n = None

            for v in open_lst:
                if n is None or poo[v] + self.h(v) < poo[n] + self.h(n):
                    n = v

            if n is None:
                print('Path does not exist!')
                return None

            if n == stop:
                reconst_path = []

                while par[n] != n:
                    reconst_path.append(n)
                    n = par[n]

                reconst_path.append(start)
                reconst_path.reverse()

                print('Path found: {}'.format(reconst_path))
                return reconst_path

            for (m, weight) in self.get_neighbors(n):
                if m not in open_lst and m not in closed_lst:
                    open_lst.add(m)
                    par[m] = n
                    poo[m] = poo[n] + weight
                else:
                    if poo[m] > poo[n] + weight:
                        poo[m] = poo[n] + weight
                        par[m] = n

                        if m in closed_lst:
                            closed_lst.remove(m)
                            open_lst.add(m)

            open_lst.remove(n)
            closed_lst.add(n)

        print('Path does not exist!')
        return None

adjac_lis = {
    'A': [('B', 1), ('C', 3), ('D', 7)],
    'B': [('D', 5)],
    'C': [('D', 12)]
}

graph1 = Graph(adjac_lis)
graph1.a_star_algorithm('A', 'D')


Path found: ['A', 'B', 'D']


['A', 'B', 'D']

Đồ thị có trọng số, được biểu diễn bằng danh sách kề:

'A': [('B', 1), ('C', 3), ('D', 7)]
'B': [('D', 5)]
'C': [('D', 12)]


Mục tiêu: tìm đường đi chi phí nhỏ nhất từ A đến D.

2. Heuristic trong code
def h(self, n):
    H = {'A':1,'B':1,'C':1,'D':1}


Heuristic h(n) = 1 cho mọi node.

Nghĩa là heuristic không phân biệt node nào gần hay xa đích.

Khi đó:

A* gần như Dijkstra, vì so sánh chủ yếu dựa vào g(n).

3. Các cấu trúc dữ liệu chính
3.1. Open list
open_lst = set([start])


Chứa các node đã phát hiện nhưng chưa mở rộng.

Node trong open list sẽ được xét tiếp.

3.2. Closed list
closed_lst = set([])


Chứa các node đã mở rộng xong.

Tránh duyệt lại node không cần thiết.

3.3. Chi phí g(n)
poo[start] = 0


poo[n] lưu chi phí thực từ start đến node n.

Đây chính là g(n) trong A*.

3.4. Parent map
par[start] = start


Lưu node cha để truy vết đường đi khi tới đích.

4. Cách A* chọn node để mở rộng
if poo[v] + self.h(v) < poo[n] + self.h(n):
    n = v


Giá trị so sánh là:

f(n)=g(n)+h(n)

Node nào có f(n) nhỏ nhất sẽ được chọn mở rộng trước.

Đây là điểm cốt lõi của A*.

5. Khi nào thuật toán dừng?
if n == stop:


Khi node được lấy ra từ open list chính là node đích.

Lúc này A* đảm bảo:

Đường đi tìm được là tối ưu (do heuristic không vượt quá chi phí thực).

6. Cách cập nhật các node kề
for (m, weight) in self.get_neighbors(n):


Xét từng node kề m của n:

Trường hợp 1: Node mới
if m not in open_lst and m not in closed_lst:


Thêm m vào open list.

Gán cha par[m] = n.

Cập nhật chi phí:
g(m)=g(n)+weight

g(m)=g(n)+weight
Trường hợp 2: Đã tồn tại nhưng tìm được đường tốt hơn
if poo[m] > poo[n] + weight:


Cập nhật lại chi phí g(m).

Cập nhật lại cha.

Nếu node đang ở closed list → đưa lại vào open list.

Điều này giúp A* luôn giữ đường đi tốt nhất.

7. Kết quả với dữ liệu đã cho

Các đường đi từ A đến D:

A → D = 7

A → B → D = 1 + 5 = 6

A → C → D = 3 + 12 = 15

A* chọn:

Path found: ['A', 'B', 'D']


Vì đây là đường có chi phí nhỏ nhất.

8. Bản chất A* trong code này (kết luận chuẩn bài)

Thuật toán sử dụng:

g(n): chi phí thực

h(n): heuristic

Luôn mở rộng node có f(n) = g(n) + h(n) nhỏ nhất.

Với heuristic bằng nhau, A* hoạt động tương đương Dijkstra.

Đảm bảo tìm được đường đi tối ưu trên đồ thị có trọng số.

So sánh AKT và A*


Giống nhau

Đều là thuật toán tìm kiếm có hướng.

Đều sử dụng heuristic để đánh giá trạng thái/node.

Đều lưu trạng thái cha để truy vết lời giải.

Đều dừng khi đạt tới trạng thái hoặc node đích.

AKT (8-puzzle)

Áp dụng cho bài toán không gian trạng thái.

Trạng thái được biểu diễn bằng ma trận 3×3.

Hàm đánh giá chỉ dựa trên heuristic (số ô sai vị trí).

Không xét chi phí đường đi đã thực hiện.

Thuật toán đơn giản, mang tính đặc thù.

A*

Áp dụng cho đồ thị có trọng số.

Đánh giá node bằng hàm f(n) = g(n) + h(n).

Kết hợp chi phí thực và heuristic.

Sử dụng tập open và closed để quản lý node.

Đảm bảo tìm được đường đi tối ưu.

Kết luận

AKT phù hợp với các bài toán trạng thái như 8-puzzle.

A* tổng quát hơn và chính xác hơn trong bài toán tìm đường.

